In [1]:
 from langchain_community.document_loaders import PyPDFLoader
 loader= PyPDFLoader("Image_Processing_Technology_Based_on_Machine_Learning.pdf")
 data = loader.load()

In [3]:
len(data)

6

In [4]:
data

[Document(metadata={'producer': 'Microsoft® Word 2013; modified using iText® 7.1.1 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Appligent AppendPDF Pro 5.5', 'creationdate': '2022-02-11T10:30:48-05:00', 'moddate': '2022-02-11T11:27:56-05:00', 'ieee article id': '9712416', 'ieee issue id': '9085929', 'subject': 'IEEE Consumer Electronics Magazine; ;PP;99;10.1109/MCE.2022.3150659', 'ieee publication id': '5962380', 'title': 'Image Processing Technology Based on Machine Learning', 'appligent': 'AppendPDF Pro 5.5 Linux Kernel 2.6 64bit Oct  2 2014 Library 10.1.0', 'source': 'Image_Processing_Technology_Based_on_Machine_Learning.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content="2162-2248 (c) 2021 IEEE. Personal use is permitted, but republication/redistribution requires IEEE permission. See http://www.ieee.org/publications_standards/publications/rights/index.html for more information.\nThis article has been accepted for publication in a future issue of this jour

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data) 
print("Total number of documents:", len(texts))

Total number of documents: 50


In [7]:
docs[4]

Document(metadata={'producer': 'Microsoft® Word 2013; modified using iText® 7.1.1 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Appligent AppendPDF Pro 5.5', 'creationdate': '2022-02-11T10:30:48-05:00', 'moddate': '2022-02-11T11:27:56-05:00', 'ieee article id': '9712416', 'ieee issue id': '9085929', 'subject': 'IEEE Consumer Electronics Magazine; ;PP;99;10.1109/MCE.2022.3150659', 'ieee publication id': '5962380', 'title': 'Image Processing Technology Based on Machine Learning', 'appligent': 'AppendPDF Pro 5.5 Linux Kernel 2.6 64bit Oct  2 2014 Library 10.1.0', 'source': 'Image_Processing_Technology_Based_on_Machine_Learning.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='proposed a social spider optimized image fusion method, \nwhich can increase contrast while maintaining brightness \nwhile fusing images.K. Li et al. [5] used multi-peak feature \nfusion for map labeling, which can effectively improve \nlabeling accuracy. Montesinos et al. [6] use Bayesian

In [12]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("What is the main topic of the document?")
print("Vector:", vector)

Vector: [0.0216510072350502, -0.05326151102781296, -0.01706888899207115, -0.010003247298300266, 0.0028219434898346663, 0.02102307230234146, 0.005405785050243139, -0.011608060449361801, -0.010245861485600471, 0.029342474415898323, 0.027084819972515106, 0.04429858922958374, -0.012020562775433064, -0.01116825733333826, 0.041813015937805176, -0.004172212444245815, 0.03661104291677475, 0.025845151394605637, -0.02883484959602356, -0.02272123284637928, 0.01161108911037445, 0.015260900370776653, -0.028955131769180298, -0.016405928879976273, -0.01337172370404005, -0.008039526641368866, 0.002896621823310852, -0.09910140931606293, -0.047610677778720856, 0.04190785065293312, -0.08954324573278427, 0.02879638597369194, -0.048032958060503006, 0.02850363776087761, 0.02164633199572563, -0.06963147968053818, -0.014206469990313053, -0.016830584034323692, -0.04087647423148155, 0.012827912345528603, -0.020872924476861954, -0.0061292871832847595, -0.04103962332010269, -0.02347959205508232, 0.046109277755022

In [14]:
vectorstore= Chroma.from_documents(
    documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)

In [20]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("What is the main topic of the document?")

In [21]:
len(retrieved_docs)

3

In [24]:
print(retrieved_docs[1].page_content)

of machine learning is increasingly extensive. On the other 
hand, with the advancement of science and technology, 
graphics have been the an indispensable medium of 
information transmission, and image processing technology is 
also booming. However, the traditional image processing 
technology, more or less has some defects, this paper 
introduces machine learning into image processing, and 
studies the image processing technology based on machine 
learning. This paper summarizes the current popular image 
processing technology, compares various image technology in 
detail, and explains the limitations of each image processing 
method. In addition, on the basis of image processing, this 
paper introduces machine learning algorithm, applies 
convolution neural network to feature extraction of image 
processing, and carries out simulation test. In the test, we 
select voc2007 dataset for image segmentation, Imagenet 
dataset for target detection, cifar100 dataset for image


In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1, max_output_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are a helpful assistant that answers questions based on the provided context. "
    "If you don't know the answer, just say that you don't know. "
    "Don't try to make up an answer."
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What is the main topic of the document?"})
print("Response:", response["answer"])

Response: The main topic is the application of machine learning, specifically deep learning, in image processing, focusing on image segmentation, classification, and target detection.  The document discusses the limitations of traditional image processing, the advantages of using machine learning, and presents results from experiments using various datasets (VOC2007, ImageNet, CIFAR100).
